# SkySat-snow pipeline

In [1]:
import os
import sys

# Define paths in directory
code_dir = '/Users/raineyaberle/Research/PhD/SnowDEMs/skysat-snow'
ref_dem_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/Banner/refdem/Banner_Bare_Earth_DEMs_mosaic_UTM11WGS84.tif'
pc_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/Banner/20240419-1/Banner_20240419-1_pc_merged.laz'
roads_fn = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/Banner/roads/Banner_roads_line.shp'
multispec_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/Banner/20240419-1/Banner_20240419-1_4band_mosaic.tif'
asp_dir = '/Users/raineyaberle/Research/PhD/SnowDEMs/StereoPipeline-3.5.0-alpha-2024-10-05-x86_64-OSX/bin'
out_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/Skysat-Stereo/study-sites/Banner/20240419-1'
res = 2 # spatial resolution of outputs [m]
roads_buffer = 5 # buffer for roads geospatial file [m]

# Check that input files and directories exist
if not os.path.exists(ref_dem_fn):
    print('Reference DEM file not found, please correct ref_dem_fn before continuing.')
if not os.path.exists(pc_fn):
    print('Point cloud file not found, please correct pc_fn before continuing.')
if not os.path.exists(roads_fn):
    print('Roads geospatial file not found, please correct roads_fn before continuing.')
if not os.path.exists(multispec_dir):
    print('Multispectral images folder not found, please correct multispec_dir before continuing.')
if not os.path.exists(asp_dir):
    print('Path to ASP not found, please correct asp_dir before continuing.')
out_dir = os.path.join(out_dir, 'skysat_snow')
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
    print('Made directory for outputs:', out_dir)

# Make results directories
preprocess_dir = os.path.join(out_dir, 'preprocess')
masks_dir = os.path.join(out_dir, 'land_cover_masks')
deramp_dir = os.path.join(out_dir, 'deramp')
pc_align_dir = os.path.join(out_dir, 'pc_align')
pc_transform_dir = os.path.join(out_dir, 'pc_transform')
final_align_dir = os.path.join(out_dir, 'final_align')
for folder in [preprocess_dir, masks_dir, deramp_dir, pc_align_dir, pc_transform_dir, final_align_dir]:
    if not os.path.exists(folder):
        os.mkdir(folder)

# Add path to pipeline utilities
sys.path.append(os.path.join(code_dir, 'skysat-snow'))
import pipeline_utils_pc as f

## 1. Preprocessing

In [ ]:
# Merge and filter point clouds
pc_filtered_las_fn, pc_filtered_tif_fn = f.preprocess_point_clouds(pc_fn, ref_dem_fn, res, preprocess_dir)

# Mosaic 4-band SR imagery, preprocess roads and reference DEM
multispec_mosaic_fn, roads_adj_fn, ref_dem_adj_fn = f.preprocess_multispec_refdem_roads(multispec_dir, ref_dem_fn, roads_fn, roads_buffer, res, out_dir)


## 2. Construct land cover masks

In [ ]:
multispec_mosaic_fn, trees_mask_fn, snow_mask_fn, roads_mask_fn, ss_mask_fn = f.construct_land_cover_masks(multispec_mosaic_fn, 
                                                                                                           roads_adj_fn, 
                                                                                                           masks_dir, 
                                                                                                           roads_buffer,
                                                                                                           ndvi_threshold=0.2,
                                                                                                           ndsi_threshold=0.0)

## 3. Deramp the DEM raster and point cloud

In [ ]:
pc_deramped_fn = f.deramp(ref_dem_fn, pc_filtered_tif_fn, pc_filtered_las_fn, deramp_dir, vmin=-10, vmax=10)

## 4. Align and transform the point cloud

In [ ]:
dem_aligned_fn = f.align_transform_pc(asp_dir, pc_deramped_fn, ref_dem_adj_fn, res, pc_align_dir, pc_transform_dir)

## 5. Final raster alignment: sub-pixel coregistration and vertical adjustment using stable surfaces

In [ ]:
final_dem_fn, final_ddem_fn = f.raster_adjustments(ref_dem_adj_fn, dem_aligned_fn, ss_mask_fn, res, final_align_dir, vmin=-10, vmax=10)

## 6. Adjust and plot final DEM and dDEM

In [ ]:
f.plot_dem_ddem(final_dem_fn, final_ddem_fn, ss_mask_fn, os.path.join(out_dir, 'final_dem_ddem.png'), vmin=-5, vmax=5)